## Customer Actions

Este archivo tiene la intencion de servir como mediador entre el sistema y las acciones que un usario con rol de customer debe ser capaz de ejecutar. 

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))

### Crear un customer
Dentro de las posibles acciones de un customer esta crear su usuario. 
Para crear un usuario a traves de su repositorio le pasamos sus datos y este se encargara de mediar entre estos y la base de datos 

In [ ]:
from datetime import datetime
from data_access.db.db import SessionLocal
from data_access.db.models import Customer
from data_access.db.repositories import CustomerRepository
from services.logger import Logger  # Importa tu clase Customer

logger = Logger()

# Crear una instancia del repositorio
session = SessionLocal
customer_repository = CustomerRepository(session)
try: 
    # Crear un nuevo usuario
    new_customer = Customer(
        age=25,
        gender='Male',
        item_purchased='Laptop',
        category='Electronics',
        purchase_amount_usd='1200',
        location='New York',
        size='Medium',
        color='Black',
        season='Winter',
        review_rating='5',
        subscription_status='Active',
        shipping_type='Standard',
        discount_applied='10%',
        promo_code_used='SAVE10',
        previous_purchases='Smartphone, Tablet',
        payment_method='Credit Card',
        frequency_of_purchases='Monthly'
    )

    customer_repository.add(new_customer)
    logger.info(f"User created successfully with id: {new_customer.customer_id}")

except Exception as e:
    session.rollback() 
    logger.error(f"An error occurred while creating the user: {str(e)}")

finally: 
    # Cerrar la sesión
    session.close()

### Leer productos
EL usuario es capaz leer los productos disponibles para sus interacciones con ellos. 

In [ ]:
from data_access.db.repositories import ProductRepository
from data_access.db.db import SessionLocal
from services.logger import Logger

logger = Logger()

session = SessionLocal()
product_repository = ProductRepository(session)
products = product_repository.get_all_paginated(page=1, page_size=10)

table_data = []
for product in products:
    table_data.append([product.unique_id, product.product_name, product.about_product])

logger.print_markdown_table(["Id", "Name", "About"],table_data)

### Interactuar
Con el fin de poder usar la tecnica de filtrado colaborativo es necesario llevar un registro de todas las interacciones del cliente con nuestros productos, estos datos seran utilizados para las recomendaciones que se le presentaran 

In [ ]:
from data_access.db.repositories import InteractionRepository
from data_access.db.db import SessionLocal

session = SessionLocal
interaction_repository = InteractionRepository(session)

user_id = 3093
product_id = '66d49bbed043f5be260fa9f7fbff5957'
# Interaction types: 'view' 'like' 'purchase'
interaction_type = 'view'
description = 'User viewed the product'

interaction = interaction_repository.create_interaction(user_id, product_id, interaction_type, description)

### Obtener recomendaciones


In [ ]:
from filters.collaborative_filter import CollaborativeFilter
from data_access.db.db import SessionLocal
from filters.content_based_filter import ContentBaseFilter
from filters.filter_pipe import FilterPipe
from models.context_model import Context
from data_access.db.repositories import ProductRepository

session = SessionLocal
filter_pipe = FilterPipe([ContentBaseFilter(session),CollaborativeFilter(session)])

# filter = CollaborativeFilter(session)
# filter = ContentBaseFilter(session)

products = ProductRepository(session).get_all_paginated(1,10)

user_id = 3094
context = Context(products,user_id,10)

result = filter_pipe.apply_filters(context)
result.show_recommendations()
